<a href="https://colab.research.google.com/github/visumania/DeepSexist/blob/main/DatasetManagement/Dataset_Task_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Montura de google colab para poder leer/escribir en Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importación de librerías necesarias

In [2]:
import pandas as pd

# Fichero de entrenamiento

Lectura y tratamiento de los datos del fichero de entrenamiento

In [3]:
# df = pd.read_json('/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training.json', orient='index') # Lectura del dataset desde Drive
df = pd.read_json('/content/drive/MyDrive/dataset/EXIST 2025 Videos Dataset/training/EXIST2025_training.json', orient='index') # Lectura del dataset desde Drive

df.reset_index(drop=True, inplace=True) # Reseteamos el index del dataframe
df = df.drop(columns=["id_Tiktok", "video", "url", "annotators", "gender_annotators", "labels_task3_2", "split"]) # Eliminación de columnas que no son relevantes del dataframe

In [4]:
df

,id_EXIST,lang,text,path_video,number_annotators,labels_task3_1,labels_task3_3
0,120001,es,cuando ves a las del 08 en la fiesta tu amigo...,videos/7281385962049998086.mp4,3,"[YES, NO, YES]","[[OBJECTIFICATION], [-], [OBJECTIFICATION]]"
1,120002,es,mujer sola caminando por la calle | yo automat...,videos/7164058026352168197.mp4,2,"[YES, YES]","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [MISOGYNY-NON..."
2,120003,es,mi amigo no le importa ni las mujeres ni las ...,videos/7248606026386263323.mp4,2,"[NO, NO]","[[-], [-]]"
3,120004,es,confirman las chicas cogiendo confianza despué...,videos/7305803156074597665.mp4,3,"[NO, YES, YES]","[[-], [STEREOTYPING-DOMINANCE], [STEREOTYPING-..."
4,120005,es,aplastar la realidad la gordita del salón tien...,videos/7318400739775204614.mp4,2,"[YES, YES]","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN..."
...,...,...,...,...,...,...,...
2519,121520,es,temach le contesta a juan guarnizo 'compa_revj...,videos/7200979592717339910.mp4,2,"[YES, YES]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION]]"
2520,121521,es,"sígueme, persona, rk, nik, meuk, ad0, mefor, ...",videos/7211806336047156486.mp4,2,"[NO, NO]","[[-], [-]]"
2521,121522,es,vas a seguir regalando flores? no regales ni a...,videos/7213218074936315141.mp4,2,"[YES, YES]","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN..."
2522,121523,es,héctor dice que el sábado puse un pedo en cas...,videos/7217265848409492782.mp4,2,"[NO, NO]","[[-], [-]]"


In [5]:
# Distribución de datos importantes en el fichero

print("Distribución del número de anotadores")
print(df.value_counts("number_annotators"))

print("\nDistribución del idioma")
print(df.value_counts("lang"))

print("\nDistribución del etiquetado para la tarea 1")
print(df['labels_task3_1'].value_counts())

print("\nDistribución del etiquetado para la tarea 3")
print(df['labels_task3_3'].value_counts())

Distribución del número de anotadores
number_annotators
2    1780
3     744
Name: count, dtype: int64

Distribución del idioma
lang
es    1524
en    1000
Name: count, dtype: int64

Distribución del etiquetado para la tarea 1
labels_task3_1
[NO, NO]               1069
[YES, YES]              711
[YES, NO, YES]          255
[NO, YES, NO]           140
[YES, YES, YES]         108
[NO, YES, YES]          101
[YES, NO, NO]            62
[UNKNOWN, NO, NO]        20
[UNKNOWN, YES, YES]      19
[NO, UNKNOWN, NO]        15
[UNKNOWN, YES, NO]        7
[UNKNOWN, NO, YES]        6
[YES, UNKNOWN, YES]       4
[YES, YES, NO]            4
[YES, UNKNOWN, NO]        2
[NO, UNKNOWN, YES]        1
Name: count, dtype: int64

Distribución del etiquetado para la tarea 3
labels_task3_3
[[-], [-]]                                                                                             1069
[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMINANCE]]                                                    234
[[STEREOTY

Tratamiento del fichero orifinal para generar uno nuevo en donde solamente se recoja la información relevante para la tarea 3.1

La primera tarea es una clasificación binaria. Los sistemas tienen que decidir si un vídeo dado contiene o no expresiones o comportamientos sexistas (es decir, es sexista en sí mismo, describe una situación sexista o critica un comportamiento sexista), y clasificarlos según dos categorías: **YES** y **NO**

In [6]:
# Eliminamos la columna correspondiente a la tarea 3.3
df_3_1 = df.drop(columns=["labels_task3_3"])

# Expande la columna labels_task3_1 en varias columnas
df_3_1_expanded = df_3_1["labels_task3_1"].apply(pd.Series)

# Renombra las columnas resultantes para mayor claridad
df_3_1_expanded.columns = [f"labels_task_3_1_{i+1}" for i in range(df_3_1_expanded.shape[1])]

# Une con el dataframe original sin la columna original
df_3_1 = pd.concat([df_3_1.drop(columns=["labels_task3_1"]), df_3_1_expanded], axis=1)

# Pasamos a binario los valores YES y NO
df_3_1.iloc[:, 5:] = df_3_1.iloc[:, 5:].replace({"YES": int(1), "NO": int(0)})

# # Esto se hace si queremos determinar si un valor específico de una fila/columna de un dataframe es NaN
# # print(pd.isna(df_3_1.at[0, 'labels_task_3_1_3']))

df_3_1.iloc[:, 7] = df_3_1.iloc[:, 7].astype("Int64")  # Pasamos los valores binarios de la última columna a int64 --> Tipo entero que soporta NaN

# print(str(type(df_3_1.iloc[0,6]))=="<class 'pandas._libs.missing.NAType'>")

print(df_3_1['labels_task_3_1_1'].value_counts())
print(df_3_1['labels_task_3_1_2'].value_counts())
print(df_3_1['labels_task_3_1_3'].value_counts())

# df_3_1

labels_task_3_1_1
0          1326
1          1146
UNKNOWN      52
Name: count, dtype: int64
labels_task_3_1_2
0          1412
1          1090
UNKNOWN      22
Name: count, dtype: int64
labels_task_3_1_3
1    494
0    250
Name: count, dtype: int64


/tmp/ipython-input-6-3343910720.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_3_1.iloc[:, 5:] = df_3_1.iloc[:, 5:].replace({"YES": int(1), "NO": int(0)})


Unificado de las anotaciones de los diferentes anotadores/as por mayoría: Si tiene 3 anotadores se unifica en el etiquetado que haya predominado en cada anotador/a.
Si hay 2 anotadores/as y conflictos habrá que decidir si dejarlo como 0 o 1

In [7]:
df_3_1["label_task_3_1_merged"] = 0 # Creación de una nueva columna en donde recogeremos la unión de la etiquetas impuestas por los/las anotadores/as

In [8]:
print(str(df_3_1.iloc[1]['labels_task_3_1_3']) == '<NA>')
print(df_3_1.iloc[0]['labels_task_3_1_1'] == 'UNKNOWN')

True
False


In [9]:
for i in range(df_3_1.shape[0]):
  votacionPositiva = votacionNegativa = 0
  numeroAnotadores = df_3_1.iloc[i]['number_annotators'] # Asimilamos que el número de anotadores es el que nos dice la columna pero lo verificaremos en las siguientes filas de código

  if df_3_1.iloc[i]['labels_task_3_1_1'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_1'] == 1:
    votacionPositiva += 1
  elif df_3_1.iloc[i]['labels_task_3_1_1'] == 'UNKNOWN': # En caso de que el etiquetado sea UNKNOWN (contabiliza al anotador pero no nos aporta información por lo que podemos descartarlo)
    numeroAnotadores -= 1

  if df_3_1.iloc[i]['labels_task_3_1_2'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_2'] == 1:
    votacionPositiva += 1
  elif df_3_1.iloc[i]['labels_task_3_1_2'] == 'UNKNOWN': # En caso de que el etiquetado sea UNKNOWN (contabiliza al anotador pero no nos aporta información por lo que podemos descartarlo)
    numeroAnotadores -= 1

  if str(df_3_1.iloc[i]['labels_task_3_1_3']) == '0':
    votacionNegativa += 1
  elif str(df_3_1.iloc[i]['labels_task_3_1_3']) == '1':
    votacionPositiva += 1

  if numeroAnotadores == 3:
    if votacionPositiva > votacionNegativa:
      df_3_1.at[i, 'label_task_3_1_merged'] = 1
    elif votacionPositiva < votacionNegativa:
      df_3_1.at[i, 'label_task_3_1_merged'] = 0

  if numeroAnotadores == 2:
    if votacionPositiva == 2:
      df_3_1.at[i, 'label_task_3_1_merged'] = 1
    elif votacionNegativa == 2:
      df_3_1.at[i, 'label_task_3_1_merged'] = 0
    elif votacionNegativa == votacionPositiva == 1:
      df_3_1.at[i, 'label_task_3_1_merged'] = 'To_delete' # CONFLICTO. Tenemos 3 opciones etiquetar 0, etiquetar 1 o descartar la fila

# df_3_1

/tmp/ipython-input-9-280604306.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'To_delete' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_3_1.at[i, 'label_task_3_1_merged'] = 'To_delete' # CONFLICTO. Tenemos 3 opciones etiquetar 0, etiquetar 1 o descartar la fila


In [10]:
df_3_1 = df_3_1.drop(columns=["number_annotators","labels_task_3_1_1", "labels_task_3_1_2", "labels_task_3_1_3"])

In [13]:
# Distribución del etiquetado después del tratado inicial de los datos
print(df_3_1['label_task_3_1_merged'].value_counts())

label_task_3_1_merged
0    1306
1    1202
Name: count, dtype: int64


In [12]:
# Eliminación de las filas del dataframe que son conflictivas (2 anotadores con 2 etiquetas distintas)

df_3_1 = df_3_1.drop(df_3_1[df_3_1['label_task_3_1_merged'] == 'To_delete'].index)
df_3_1.reset_index(drop=True, inplace=True)

In [14]:
df_3_1

,id_EXIST,lang,text,path_video,label_task_3_1_merged
0,120001,es,cuando ves a las del 08 en la fiesta tu amigo...,videos/7281385962049998086.mp4,1
1,120002,es,mujer sola caminando por la calle | yo automat...,videos/7164058026352168197.mp4,1
2,120003,es,mi amigo no le importa ni las mujeres ni las ...,videos/7248606026386263323.mp4,0
3,120004,es,confirman las chicas cogiendo confianza despué...,videos/7305803156074597665.mp4,1
4,120005,es,aplastar la realidad la gordita del salón tien...,videos/7318400739775204614.mp4,1
...,...,...,...,...,...
2503,121520,es,temach le contesta a juan guarnizo 'compa_revj...,videos/7200979592717339910.mp4,1
2504,121521,es,"sígueme, persona, rk, nik, meuk, ad0, mefor, ...",videos/7211806336047156486.mp4,0
2505,121522,es,vas a seguir regalando flores? no regales ni a...,videos/7213218074936315141.mp4,1
2506,121523,es,héctor dice que el sábado puse un pedo en cas...,videos/7217265848409492782.mp4,0


In [15]:
# Guardamos el fichero resultante en nuestro Drive
# df_3_1.to_csv('/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv', index=True)
df_3_1.to_csv('/content/drive/MyDrive/dataset/EXIST 2025 Videos Dataset/training/EXIST2025_training_3_1.csv', index=True)

# Fichero de test

Solamente realizo unas pruebas para ver el contenido con el que viene, pero poco se puede hacer a nivel de gestión de datos porque, como es obvio, no viene etiquetado.

In [ ]:
df = pd.read_json('/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/test/EXIST2025_test_clean.json', orient='index') # Lectura del dataset desde Drive
df.reset_index(drop=True, inplace=True) # Reseteamos el index del dataframe
df = df.drop(columns=["id_Tiktok", "video", "url", "annotators", "gender_annotators", "split"]) # Eliminación de columnas que no son relevantes del dataframe

df

,id_EXIST,lang,text,path_video,number_annotators
0,320001,es,si me siguió te crece +3cm me echan del instit...,videos/7295500749532925217.mp4,2
1,320002,es,humor gente un cardumen de mujeres en su hábit...,videos/7122933542853676294.mp4,2
2,320003,es,"auronplay diciendo ""un generador de niños, que...",videos/7290197397878115617.mp4,2
3,320004,es,no se vé face_with_peeking_eye mi amigo reacc...,videos/7208714038174305542.mp4,2
4,320005,es,"¿y las promesas? siempre mienten, fin. el tie...",videos/7311111177982053638.mp4,2
...,...,...,...,...,...
669,320300,es,no somos así estamos para que hagan un filtro ...,videos/7290998333789818118.mp4,3
670,320301,es,"no eres salvavidas de nadie, sólo de ti misma ...",videos/7268727677312322821.mp4,2
671,320302,es,amate como el temach y ama como farid dieck ...,videos/7289596067992145158.mp4,2
672,320303,es,dos filosofías muy diferentes pero q hablan de...,videos/7288127010029751558.mp4,2
